## SHREE KRISHNA KANTH S
## 225229136
## II MSc DATA SCIENCE "A"
## PDL LAB 17

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, MaxPooling1D, LSTM, Dense, Dropout
from keras.utils import to_categorical
from sklearn.datasets import fetch_20newsgroups

In [2]:
# Step 1: Import libraries and open dataset
# Assuming you have already loaded your dataset into a variable called 'data'
data_full = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'), shuffle=True, random_state=42)
data = pd.DataFrame()
data['text'] = data_full.data
data['source'] = data_full.target
label=[]
for i in data['source']:
    label.append(data_full.target_names[i])
data['label']=label

In [3]:
data

,text,source,label
0,\n\nI am sure some bashers of Pens fans are pr...,10,rec.sport.hockey
1,My brother is in the market for a high-perform...,3,comp.sys.ibm.pc.hardware
2,\n\n\n\n\tFinally you said what you dream abou...,17,talk.politics.mideast
3,\nThink!\n\nIt's the SCSI card doing the DMA t...,3,comp.sys.ibm.pc.hardware
4,1) I have an old Jasmine drive which I cann...,4,comp.sys.mac.hardware
...,...,...,...
18841,DN> From: nyeda@cnsvax.uwec.edu (David Nye)\nD...,13,sci.med
18842,\nNot in isolated ground recepticles (usually ...,12,sci.electronics
18843,I just installed a DX2-66 CPU in a clone mothe...,3,comp.sys.ibm.pc.hardware
18844,\nWouldn't this require a hyper-sphere. In 3-...,1,comp.graphics


In [4]:
# Step 2: Pre-processing the Text
data['text'] = data['text'].str.lower()
data['text'] = data['text'].str.replace('[^a-zA-Z\s]', '')

C:\Users\shree\AppData\Local\Temp\ipykernel_2080\430632324.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  data['text'] = data['text'].str.replace('[^a-zA-Z\s]', '')


In [5]:
# Step 3: Dataset Preparation
X = data['text']
y = data['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [6]:
# Step 4: LSTM Model Creation
max_words = 10000  # You can adjust this based on your dataset
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

max_sequence_length = 100  # You can adjust this based on your dataset
X_train_pad = pad_sequences(X_train_seq, maxlen=max_sequence_length)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_sequence_length)

# Modify this section to convert labels to integer labels
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Convert integer labels to one-hot encoding
from keras.utils import to_categorical
y_train_encoded = to_categorical(y_train_encoded)
y_test_encoded = to_categorical(y_test_encoded)

model_lstm = Sequential()
model_lstm.add(Embedding(input_dim=max_words, output_dim=100, input_length=max_sequence_length))
model_lstm.add(LSTM(100))
model_lstm.add(Dense(64, activation='relu'))
model_lstm.add(Dense(len(label_encoder.classes_), activation='softmax'))  # Adjust the number of units based on your labels
model_lstm.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [7]:
model_lstm.fit(X_train_pad, y_train_encoded, epochs=20, batch_size=64)
loss, accuracy = model_lstm.evaluate(X_test_pad, y_test_encoded)
print(f"LSTM Model - Loss: {loss}, Accuracy: {accuracy}")

Epoch 1/20
207/207 [==============================] - 18s 80ms/step - loss: 2.7692 - accuracy: 0.1259
Epoch 2/20
207/207 [==============================] - 17s 81ms/step - loss: 2.0167 - accuracy: 0.3025
Epoch 3/20
207/207 [==============================] - 17s 82ms/step - loss: 1.4677 - accuracy: 0.4910
Epoch 4/20
207/207 [==============================] - 17s 83ms/step - loss: 1.0933 - accuracy: 0.6359
Epoch 5/20
207/207 [==============================] - 17s 83ms/step - loss: 0.8084 - accuracy: 0.7379
Epoch 6/20
207/207 [==============================] - 17s 84ms/step - loss: 0.6238 - accuracy: 0.8044
Epoch 7/20
207/207 [==============================] - 17s 83ms/step - loss: 0.4715 - accuracy: 0.8576
Epoch 8/20
207/207 [==============================] - 17s 83ms/step - loss: 0.4291 - accuracy: 0.8709
Epoch 9/20
207/207 [==============================] - 17s 84ms/step - loss: 0.3367 - accuracy: 0.8999
Epoch 10/20
207/207 [==============================] - 18s 86ms/step - loss: 0.258

In [8]:
# Step 5: CNN-LSTM Model Creation
model_cnn_lstm = Sequential()
model_cnn_lstm.add(Embedding(input_dim=max_words, output_dim=100, input_length=max_sequence_length))
model_cnn_lstm.add(Conv1D(128, 5, activation='relu'))
model_cnn_lstm.add(MaxPooling1D(5))
model_cnn_lstm.add(LSTM(100))
model_cnn_lstm.add(Dense(64, activation='relu'))
model_cnn_lstm.add(Dense(len(y.unique()), activation='softmax'))
model_cnn_lstm.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model_cnn_lstm.fit(X_train_pad, y_train_encoded, epochs=20, batch_size=64)
loss, accuracy = model_cnn_lstm.evaluate(X_test_pad, y_test_encoded)
print(f"CNN-LSTM Model - Loss: {loss}, Accuracy: {accuracy}")

Epoch 1/20
207/207 [==============================] - 10s 38ms/step - loss: 2.6375 - accuracy: 0.1302
Epoch 2/20
207/207 [==============================] - 8s 38ms/step - loss: 1.8366 - accuracy: 0.3477
Epoch 3/20
207/207 [==============================] - 8s 38ms/step - loss: 1.2378 - accuracy: 0.5776
Epoch 4/20
207/207 [==============================] - 8s 38ms/step - loss: 0.8982 - accuracy: 0.7018
Epoch 5/20
207/207 [==============================] - 8s 38ms/step - loss: 0.6837 - accuracy: 0.7799
Epoch 6/20
207/207 [==============================] - 8s 38ms/step - loss: 0.5367 - accuracy: 0.8307
Epoch 7/20
207/207 [==============================] - 8s 38ms/step - loss: 0.4245 - accuracy: 0.8692
Epoch 8/20
207/207 [==============================] - 8s 38ms/step - loss: 0.3390 - accuracy: 0.8983
Epoch 9/20
207/207 [==============================] - 8s 39ms/step - loss: 0.2704 - accuracy: 0.9206
Epoch 10/20
207/207 [==============================] - 8s 38ms/step - loss: 0.2377 - accur

In [9]:
# Step 6: Pre-trained Model Creation
# Download and load pre-trained GloVe embeddings
embedding_file = 'C:\\Users\\shree\\Downloads\\archive (5)\\glove.6B.100d.txt'

embedding_matrix = {}
with open(embedding_file, 'r', encoding='utf-8') as f:
    for line in f:
        values = line.strip().split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embedding_matrix[word] = coefs

embedding_dim = 100  # Should match the dimension of the GloVe embeddings
word_index = tokenizer.word_index
num_words = min(max_words, len(word_index) + 1)

embedding_matrix = np.zeros((num_words, embedding_dim))
for word, i in word_index.items():
    if i >= max_words:
        continue
    embedding_vector = embedding_matrix[i]
    if word in embedding_matrix:
        embedding_matrix[i] = embedding_vector

model_pretrained = Sequential()
model_pretrained.add(Embedding(input_dim=num_words, output_dim=embedding_dim, input_length=max_sequence_length,
                              weights=[embedding_matrix], trainable=False))
model_pretrained.add(Conv1D(128, 5, activation='relu'))
model_pretrained.add(MaxPooling1D(5))
model_pretrained.add(LSTM(100))
model_pretrained.add(Dense(64, activation='relu'))
model_pretrained.add(Dense(len(y.unique()), activation='softmax'))
model_pretrained.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model_pretrained.fit(X_train_pad, y_train_encoded, epochs=20, batch_size=64)
loss, accuracy = model_pretrained.evaluate(X_test_pad, y_test_encoded)
print(f"Pre-trained Model - Loss: {loss}, Accuracy: {accuracy}")

C:\Users\shree\AppData\Local\Temp\ipykernel_2080\168645916.py:22: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if word in embedding_matrix:


Epoch 1/20
207/207 [==============================] - 8s 27ms/step - loss: 2.9944 - accuracy: 0.0479
Epoch 2/20
207/207 [==============================] - 5s 26ms/step - loss: 2.9922 - accuracy: 0.0544
Epoch 3/20
207/207 [==============================] - 6s 27ms/step - loss: 2.9912 - accuracy: 0.0538
Epoch 4/20
207/207 [==============================] - 6s 27ms/step - loss: 2.9907 - accuracy: 0.0531
Epoch 5/20
207/207 [==============================] - 6s 29ms/step - loss: 2.9904 - accuracy: 0.0544
Epoch 6/20
207/207 [==============================] - 6s 29ms/step - loss: 2.9902 - accuracy: 0.0544
Epoch 7/20
207/207 [==============================] - 7s 32ms/step - loss: 2.9902 - accuracy: 0.0531
Epoch 8/20
207/207 [==============================] - 6s 30ms/step - loss: 2.9901 - accuracy: 0.0544
Epoch 9/20
207/207 [==============================] - 6s 31ms/step - loss: 2.9901 - accuracy: 0.0544
Epoch 10/20
207/207 [==============================] - 7s 32ms/step - loss: 2.9901 - accura

In [10]:
# Step 7: Improvements
# You can try different dropout rates and test different splits for further improvements.

In [11]:
# Step 7: Improvements

# 7.1 Dropout in LSTM Model
model_lstm_with_dropout = Sequential()
model_lstm_with_dropout.add(Embedding(input_dim=max_words, output_dim=100, input_length=max_sequence_length))
model_lstm_with_dropout.add(LSTM(100))
model_lstm_with_dropout.add(Dropout(0.2))  # Add dropout layer with 20% dropout rate
model_lstm_with_dropout.add(Dense(64, activation='relu'))
model_lstm_with_dropout.add(Dense(len(y.unique()), activation='softmax'))
model_lstm_with_dropout.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model_lstm_with_dropout.fit(X_train_pad, y_train_encoded, epochs=20, batch_size=64)
loss, accuracy = model_lstm_with_dropout.evaluate(X_test_pad, y_test_encoded)
print(f"LSTM Model with Dropout - Loss: {loss}, Accuracy: {accuracy}")

Epoch 1/20
207/207 [==============================] - 20s 88ms/step - loss: 2.7216 - accuracy: 0.1110
Epoch 2/20
207/207 [==============================] - 18s 87ms/step - loss: 2.0979 - accuracy: 0.2774
Epoch 3/20
207/207 [==============================] - 17s 84ms/step - loss: 1.5782 - accuracy: 0.4430
Epoch 4/20
207/207 [==============================] - 17s 84ms/step - loss: 1.2038 - accuracy: 0.5922
Epoch 5/20
207/207 [==============================] - 17s 82ms/step - loss: 0.9362 - accuracy: 0.6872
Epoch 6/20
207/207 [==============================] - 17s 82ms/step - loss: 0.7635 - accuracy: 0.7510
Epoch 7/20
207/207 [==============================] - 17s 82ms/step - loss: 0.6440 - accuracy: 0.7899
Epoch 8/20
207/207 [==============================] - 16s 79ms/step - loss: 0.5189 - accuracy: 0.8361
Epoch 9/20
207/207 [==============================] - 16s 79ms/step - loss: 0.4354 - accuracy: 0.8681
Epoch 10/20
207/207 [==============================] - 16s 78ms/step - loss: 0.337

In [12]:
# 7.2 Different Data Split
X_train_new, X_test_new, y_train_new, y_test_new = train_test_split(X, y, test_size=0.2, random_state=42)

# Re-tokenize and re-pad based on the new split
tokenizer.fit_on_texts(X_train_new)
X_train_seq_new = tokenizer.texts_to_sequences(X_train_new)
X_test_seq_new = tokenizer.texts_to_sequences(X_test_new)
X_train_pad_new = pad_sequences(X_train_seq_new, maxlen=max_sequence_length)
X_test_pad_new = pad_sequences(X_test_seq_new, maxlen=max_sequence_length)

# Encode the labels using LabelEncoder
label_encoder = LabelEncoder()
y_train_encoded_new = label_encoder.fit_transform(y_train_new)
y_test_encoded_new = label_encoder.transform(y_test_new)

# Convert integer labels to one-hot encoding
y_train_encoded_new = to_categorical(y_train_encoded_new)
y_test_encoded_new = to_categorical(y_test_encoded_new)

model_cnn_lstm_split = Sequential()
model_cnn_lstm_split.add(Embedding(input_dim=max_words, output_dim=100, input_length=max_sequence_length))
model_cnn_lstm_split.add(Conv1D(128, 5, activation='relu'))
model_cnn_lstm_split.add(MaxPooling1D(5))
model_cnn_lstm_split.add(LSTM(100))
model_cnn_lstm_split.add(Dense(64, activation='relu'))
model_cnn_lstm_split.add(Dense(len(label_encoder.classes_), activation='softmax'))  # Adjust the number of units based on your labels
model_cnn_lstm_split.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model_cnn_lstm_split.fit(X_train_pad_new, y_train_encoded_new, epochs=20, batch_size=64)
loss, accuracy = model_cnn_lstm_split.evaluate(X_test_pad_new, y_test_encoded_new)
print(f"CNN-LSTM Model with Different Split - Loss: {loss}, Accuracy: {accuracy}")


Epoch 1/20
236/236 [==============================] - 12s 41ms/step - loss: 2.6023 - accuracy: 0.1510
Epoch 2/20
236/236 [==============================] - 9s 40ms/step - loss: 1.7138 - accuracy: 0.4026
Epoch 3/20
236/236 [==============================] - 9s 39ms/step - loss: 1.1753 - accuracy: 0.6066
Epoch 4/20
236/236 [==============================] - 9s 39ms/step - loss: 0.8304 - accuracy: 0.7345
Epoch 5/20
236/236 [==============================] - 10s 41ms/step - loss: 0.6310 - accuracy: 0.8012
Epoch 6/20
236/236 [==============================] - 10s 40ms/step - loss: 0.5010 - accuracy: 0.8460
Epoch 7/20
236/236 [==============================] - 10s 41ms/step - loss: 0.3924 - accuracy: 0.8793
Epoch 8/20
236/236 [==============================] - 10s 41ms/step - loss: 0.3135 - accuracy: 0.9089
Epoch 9/20
236/236 [==============================] - 11s 47ms/step - loss: 0.2681 - accuracy: 0.9197
Epoch 10/20
236/236 [==============================] - 10s 41ms/step - loss: 0.2280 -

In [13]:
# Step 8: Predictions
sample_text = "Australia claimed the crucial wicket of Sri Lankan opener Pathum Nissanka..."
sample_text = sample_text.lower().replace('[^a-zA-Z\s]', '')
sample_sequence = tokenizer.texts_to_sequences([sample_text])
sample_sequence = pad_sequences(sample_sequence, maxlen=max_sequence_length)
predicted_class = np.argmax(model_pretrained.predict(sample_sequence), axis=-1)
print(f"Predicted class: {predicted_class}")

1/1 [==============================] - 0s 365ms/step
Predicted class: [8]
